In [1]:
from IPython.display import display, Math, Markdown
from ipywidgets import interact, widgets
from langchain_groq import ChatGroq
import matplotlib.pyplot as plt
import sympy as sp

# Initialize LLM
llm = ChatGroq(model="llama3-8b-8192")

def render_math(formula):
    """Render LaTeX formula in Jupyter Notebook with enhanced display"""
    display(Math(formula))
    return formula

def latex_to_image(latex_str, filename='math_expression.png', dpi=300, format='png'):
    """
    Convert LaTeX string to an image file
    
    Args:
        latex_str: LaTeX mathematical expression
        filename: Output filename (default: 'math_expression.png')
        dpi: Image resolution
        format: 'png' or 'jpg'
    """
    # Set up matplotlib figure with white background
    fig = plt.figure(figsize=(6, 2), dpi=dpi)
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(1)
    ax = fig.add_subplot(111)
    ax.axis('off')
    
    # Render LaTeX expression
    ax.text(0.5, 0.5, f'${latex_str}$', 
            size=20, ha='center', va='center')
    
    # Save to file
    plt.savefig(filename, bbox_inches='tight', pad_inches=0.1, dpi=dpi, format=format)
    plt.close()
    
    # Display confirmation
    display(Markdown(f"**Saved as:** `{filename}`"))
    return filename

def nl_to_math(natural_language, explain=False, plot=False, save_as=None):
    """
    Enhanced natural language to LaTeX converter for Jupyter
    
    Args:
        natural_language: Math expression in natural language
        explain: Whether to show explanation
        plot: Whether to attempt plotting (for plottable expressions)
        save_as: None, 'png', or 'jpg' to save the expression as an image
    """
    # Get LaTeX conversion
    response = llm.invoke(
        f"Convert this to LaTeX code only (no additional text): {natural_language}"
    )
    latex_code = response.content.strip('`').replace('$', '')
    
    # Display results
    display(Markdown(f"**Input:** `{natural_language}`"))
    rendered = render_math(latex_code)
    
    if explain:
        explanation = llm.invoke(
            f"Explain this mathematical expression in simple terms: {latex_code}"
        )
        display(Markdown(f"**Explanation:** {explanation.content}"))
    
    if plot and any(term in natural_language.lower() for term in ['graph', 'plot', 'function']):
        try:
            plot_expression(latex_code)
        except:
            display(Markdown("*Could not generate plot for this expression*"))
    
    if save_as in ['png', 'jpg']:
        filename = f"math_expression.{save_as}"
        latex_to_image(latex_code, filename=filename, format=save_as)
    
    return latex_code

def plot_expression(latex_code):
    """Attempt to plot the mathematical expression"""
    # Simple implementation for function plotting
    try:
        expr = sp.sympify(latex_code.split('=')[1] if '=' in latex_code else latex_code)
        if len(expr.free_symbols) == 1:
            x = list(expr.free_symbols)[0]
            sp.plot(expr, (x, -5, 5), 
                   title=f"Plot of ${latex_code}$",
                   xlabel=str(x), ylabel='f(x)')
    except:
        raise ValueError("Could not plot expression")

def interactive_converter():
    """Create an interactive widget for Jupyter"""
    style = {'description_width': 'initial'}
    
    interact(
        nl_to_math,
        natural_language=widgets.Textarea(
            value='area of circle',
            description='Math expression:',
            style=style,
            layout={'width': '500px'}
        ),
        explain=widgets.Checkbox(
            value=True,
            description='Include explanation',
            style=style
        ),
        plot=widgets.Checkbox(
            value=False,
            description='Attempt to plot',
            style=style
        ),
        save_as=widgets.Dropdown(
            options=[None, 'png', 'jpg'],
            value=None,
            description='Save as image:',
            style=style
        )
    )



In [2]:
interactive_converter()

interactive(children=(Textarea(value='area of circle', description='Math expression:', layout=Layout(width='50…